In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from snorkel import SnorkelSession
snorkel = SnorkelSession()

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Snark Tutorial").master("spark://bachmacbook:7077").getOrCreate()

In [11]:
from snorkel.models import candidate_subclass
Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

In [15]:
from snorkel.models.views import create_serialized_candidate_view

create_serialized_candidate_view(snorkel, Spouse)

Creating view...

    CREATE VIEW spouse_serialized AS
        SELECT
            candidate.id,
            candidate.split,
            span0.*, span1.*,
            sentence.*
        FROM
            candidate,
            spouse,
            span AS span0, span AS span1,
            sentence
        WHERE
            candidate.id = spouse.id
            AND sentence.id = span0.sentence_id
            AND spouse.person1_id = span0.id AND spouse.person2_id = span1.id
    


OperationalError: (sqlite3.OperationalError) table spouse_serialized already exists [SQL: u'\n    CREATE VIEW spouse_serialized AS\n        SELECT\n            candidate.id,\n            candidate.split,\n            span0.*, span1.*,\n            sentence.*\n        FROM\n            candidate,\n            spouse,\n            span AS span0, span AS span1,\n            sentence\n        WHERE\n            candidate.id = spouse.id\n            AND sentence.id = span0.sentence_id\n            AND spouse.person1_id = span0.id AND spouse.person2_id = span1.id\n    ']

In [20]:
Spouse.__tablename__

'spouse'

In [33]:
snorkel.connection().driver

AttributeError: 'Connection' object has no attribute 'driver'

In [25]:
jdbcDF = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:sqlite:///Users/bach/snorkel/tutorials/snark/snorkel.db") \
    .option("dbtable", "candidate") \
    .option("user", "") \
    .option("password", "") \
    .load()

In [26]:
jdbcDF

DataFrame[id: int, type: string, split: int, training_marginal: float]

In [27]:
jdbcDF.count()

5282

In [28]:
jdbcDF.show()

+----+------+-----+-----------------+
|  id|  type|split|training_marginal|
+----+------+-----+-----------------+
|5283|spouse|    0|       0.14229347|
|5284|spouse|    0|       0.14229347|
|5285|spouse|    0|       0.14229347|
|5286|spouse|    0|       0.14229347|
|5287|spouse|    0|              0.5|
|5288|spouse|    0|              0.5|
|5289|spouse|    0|       0.14229347|
|5290|spouse|    0|       0.14229347|
|5291|spouse|    0|              0.5|
|5292|spouse|    0|       0.14229347|
|5293|spouse|    0|       0.14229347|
|5294|spouse|    0|       0.14229347|
|5295|spouse|    0|       0.14229347|
|5296|spouse|    0|       0.14229347|
|5297|spouse|    0|              0.5|
|5298|spouse|    0|       0.14229347|
|5299|spouse|    0|       0.14229347|
|5300|spouse|    0|       0.14229347|
|5301|spouse|    0|       0.14229347|
|5302|spouse|    0|              0.5|
+----+------+-----+-----------------+
only showing top 20 rows



In [8]:
jdbcDF.rdd.map(lambda x: x['training_marginal']).reduce(lambda a, b: max(a, b))

0.6526589393615723

In [29]:
candidates = jdbcDF.rdd.setName("Candidates Test")
candidates.cache()

Candidates Test MapPartitionsRDD[41] at javaToPython at NativeMethodAccessorImpl.java:0

In [30]:
candidates.map(lambda x: x['training_marginal']).reduce(lambda a, b: max(a, b))

0.6526589393615723

In [22]:
candidates.setName("Candidates Test").cache()

Candidates Test MapPartitionsRDD[12] at javaToPython at NativeMethodAccessorImpl.java:0

In [34]:
Spouse.__name__

'Spouse'